# Tutorial: explain a tabular classifier through LORE

LORE provides interpretable explanations for the decisions made by machine learning models, particularly in situations where it might be challenging to explain the model's behavior using traditional methods. LORE is designed to offer explanations on a per-instance basis, meaning it provides insights into why a specific prediction was made for a particular data point. This is in contrast to global explanations that aim to provide an overview of a model's behavior across the entire dataset.


In this tutorial we will explain the reasoning of a random forest classifier when applied to an instance of the Adult Income dataset ( https://archive.ics.uci.edu/dataset/2/adult). This dataset contains census data from 1994 census database, the classification problem associated to such data involves the prediction of the annual income of a person, given a set of socio-economic caracteristics: 
- working class
- marital status
- occupation
- relationship
- race
- sex
- capital gain/loss
- native country 





### Dataset loading 

Lore libray has a module devoted to dataset handling. Here we use TabularDataset class to load a dataset from a csv file. A TabularDataset has a column that represent the target class (`class_name`) of the classification task  object, and has two main attributes: 
- `df` : a pandas dataframe representing the tabular data
- `descriptor` : a dictionary containing internal data related to the dataset. It is build when a TabularDataset is created, but it could also be edited. It is used by the next steps of LORE methodology to distinguish among numerical, categorical, ordinal and target features of the dataset.

In [23]:
from lore_sa.dataset import TabularDataset
import pandas as pd

# Cargar el dataset
df = pd.read_csv('test/resources/iris.csv')

# Convertir la columna target a nombres de clases
species_mapping = {0: "setosa", 1: "versicolor", 2: "virginica"}
df["target"] = df["target"].map(species_mapping)

# Guardar el nuevo dataset
df.to_csv('test/resources/iris_categorical.csv', index=False)

# Cargarlo en TabularDataset
dataset = TabularDataset.from_csv('test/resources/iris_categorical.csv', class_name="target")

dataset.df.dropna(inplace = True)
dataset.df


2025-03-18 12:17:47,695 root         INFO     test/resources/iris_categorical.csv file imported


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


The `TabularDataset` class does some data inspection and preparation when it is created starting from a text-based file. Each attribute of the data is analysed to derive type and internal statistics. The `descriptor` field is automatically updates with these information. In particular, the `descriptor` is a dictionary that contains the following keys:
- `numerical` : a dictionary containing the numerical features of the dataset and their statistics
- `categorical` : a dictionary containing the categorical features of the dataset and their statistics
- `ordinal` : (this is still not implemented) a dictionary containing the ordinal features of the dataset and their statistics
- `target` : the details of the target feature of the dataset and its statistics  

In [24]:
dataset.descriptor.keys()

dict_keys(['numeric', 'categorical', 'ordinal', 'target'])

Before proceeding with the explanation, let's do some data preprocessing by dropping a few attributes that are not relevant for the learning. 

We operate directly on the `df` attribute of the dataset, then we update the descriptor.


In [25]:
# dataset.df.drop(['fnlwgt', 'education-num'], inplace=True, axis=1)
dataset.update_descriptor()

## Model Learning

We start by training a Random Forest classifier on the Adult dataset. We will use the `df` attribute of the dataset, which is a pandas dataframe containing the tabular data. We will use the `class_name` attribute of the dataset to identify the target feature of the classification task. Since the data contains non-numeric attributes, we proceed with a preprocessing of the data to mange these attributes. We will exploit `Pipeline` class from `sklearn` to create a pipeline that applies one-hot encoding to the categorical features and label encoding to the target feature. 

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lore_sa.bbox import sklearn_classifier_bbox


def train_model(dataset: TabularDataset):
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), [0,1,2,3]),
            ('cat', OrdinalEncoder(), [])
        ]
    )
    model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, random_state=42))
    
    X_train, X_test, y_train, y_test = train_test_split(dataset.df.loc[:, 'sepal length (cm)':'petal width (cm)'].values, dataset.df['target'].values,
                test_size=0.3, random_state=42, stratify=dataset.df['target'].values)
    model.fit(X_train, y_train)
    
    return sklearn_classifier_bbox.sklearnBBox(model), X_train, X_test, y_train, y_test
    

In [27]:
bbox, X_train, X_test, y_train, y_test = train_model(dataset)

from sklearn.metrics import classification_report, accuracy_score

# Obtener predicciones en el conjunto de prueba
y_pred = bbox.bbox.predict(X_test)

# Imprimir métricas
print("🔍 Reporte de clasificación:\n")
print(classification_report(y_test, y_pred))
print(f"✅ Precisión del modelo: {accuracy_score(y_test, y_pred):.2f}")


🔍 Reporte de clasificación:

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        15
  versicolor       0.78      0.93      0.85        15
   virginica       0.92      0.73      0.81        15

    accuracy                           0.89        45
   macro avg       0.90      0.89      0.89        45
weighted avg       0.90      0.89      0.89        45

✅ Precisión del modelo: 0.89


## Explanation

Given the blackbox model, we can now explain the reasoning behind the classification of a specific instance. We will use the `LORE` class to explain the classification of the first instance of the dataset. The `explain_instance` method of the `LORE` class takes as input the instance to be explained, the blackbox model, and the dataset. It returns an explanation object that contains the explanation of the classification of the instance.

The constructor of the explanator class takes as input the blackbox model and the dataset. The `dataset` instance is used to get the information from the descriptor. In case the trainin set is not available, the descriptor could be built from the test set or by the metadata of the dataset.

In [28]:
from lore_sa.lore import TabularRandomGeneratorLore
from lore_sa.lore import TabularGeneticGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [29]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
# when
explanation = tabularLore.explain(x)
# then

In [30]:
x = dataset.df.iloc[10][:-1] # we exclude the target feature
x

sepal length (cm)    5.4
sepal width (cm)     3.7
petal length (cm)    1.5
petal width (cm)     0.2
Name: 10, dtype: object

In [31]:
def format_explanation(explanation):
    """
    Formatea la explicación de LORE en un formato legible para el dataset Iris.
    """
    result = "\n" + "="*50 + "\n"
    result += "EXPLICACIÓN DEL MODELO - CLASIFICACIÓN DE IRIS\n"
    result += "="*50 + "\n\n"
    
    # Regla principal
    rule = explanation['rule']
    especie_predicha = rule['consequence']['val']  # Extraer la especie clasificada

    result += f"✅ Condiciones necesarias para que la flor sea clasificada como '{especie_predicha}':\n"
    
    for condition in rule['premises']:
        attr = condition['attr'].replace("-", " ").capitalize()
        val = round(condition['val'], 2)  # Redondear valores
        op = condition['op']
        
        # Convertir los operadores en un formato más legible
        op_text = {
            ">": ">",
            "<": "<",
            ">=": "≥",
            "<=": "≤",
            "!=": "NO es",
            "=": "es igual a"
        }.get(op, op)
        
        result += f"  - {attr} {op_text} {val}\n"
    
    result += "\n" + "-"*50 + "\n"
    result += "🔄 Casos contrafactuales donde el modelo cambiaría la predicción de especie:\n"
    result += "-"*50 + "\n\n"

    # Casos contrafactuales
    for idx, cf in enumerate(explanation['counterfactuals'], start=1):
        especie_cambio = cf['consequence']['val']  # Extraer la especie del contrafactual

        result += f"🛑 CASO {idx}: Si se cumplen estas condiciones, la flor sería clasificada como '{especie_cambio}'\n"
        
        for condition in cf['premises']:
            attr = condition['attr'].replace("-", " ").capitalize()
            val = round(condition['val'], 2)  # Redondear valores
            op = condition['op']
            
            op_text = {
                ">": ">",
                "<": "<",
                ">=": "≥",
                "<=": "≤",
                "!=": "NO es",
                "=": "es igual a"
            }.get(op, op)
            
            result += f"  - {attr} {op_text} {val}\n"
        
        result += "\n" + "-"*50 + "\n"

    return result

# Mostrar la explicación en pantalla con formato claro
formatted_explanation = format_explanation(explanation)
print(formatted_explanation)



EXPLICACIÓN DEL MODELO - CLASIFICACIÓN DE IRIS

✅ Condiciones necesarias para que la flor sea clasificada como 'setosa':
  - Petal length (cm) ≤ 2.34
  - Sepal width (cm) > 3.35

--------------------------------------------------
🔄 Casos contrafactuales donde el modelo cambiaría la predicción de especie:
--------------------------------------------------

🛑 CASO 1: Si se cumplen estas condiciones, la flor sería clasificada como 'versicolor'
  - Petal length (cm) ≤ 2.34
  - Sepal width (cm) ≤ 3.11
  - Petal width (cm) ≤ 1.69
  - Petal width (cm) > 0.9
  - Sepal length (cm) ≤ 7.13
  - Sepal length (cm) > 5.52

--------------------------------------------------
🛑 CASO 2: Si se cumplen estas condiciones, la flor sería clasificada como 'versicolor'
  - Petal length (cm) > 2.34
  - Petal width (cm) ≤ 1.69
  - Petal width (cm) > 0.7
  - Sepal length (cm) ≤ 6.19

--------------------------------------------------
🛑 CASO 3: Si se cumplen estas condiciones, la flor sería clasificada como 'versi

### Drill down the explanation process

The example above shows the explanation of the classification of one of the instances of the dataset. The `TabularRandomGenerator` encapsulates many of the deatils of the explanation process. The class construct and explanation pipeline that is composed by the following steps:
- **Neighborhood generation**: the neighborhood of the instance to be explained is generated. The neighborhood is a synthetic dataset that is created by generating random instances around the instance to be explained. The neighborhood is used to understand the behavior of the blackbox model in the vicinity of the instance to be explained.
- **Encoding**: the neighborhood dataset is encoded using one-hot encoding. The encoded neighborhood dataset is used to train the surrogate model.
- **Surrogate model**: a surrogate model is trained on the neighborhood dataset. The surrogate model is a simpler model that approximates the behavior of the blackbox model in the neighborhood of the instance to be explained. The surrogate model is used to extract classification rules that explain the behavior of the blackbox model.
- **Rule extraction**: the classification rules are extracted from the surrogate model. The classification rules are used to explain the classification of the instance to be explained. The classification rules provide insights into the features that are important for the classification of the instance.

Each of these step can be customized separately, in order to adapt the explanation process to the specific needs of the user. For example, let's construct an explanation process by managing manually the choice of each component of the explanation pipeline.

#### Encoding / Decoding

We use a `ColumnTransformerEnc` to exploit the `TransformerMixin` interface of the `sklearn` transformers. The `ColumnTransformerEnc` class is a wrapper around the `ColumnTransformer` class that provides the `encode` and `decode` methods to encode and decode the dataset. The `encode` method applies the transformations of the transformers to the dataset, while the `decode` method applies the inverse transformations to the dataset. The `ColumnTransformerEnc` class is used to encode and decode the dataset in the explanation process. 

In [32]:
from lore_sa.encoder_decoder import ColumnTransformerEnc

tabular_enc = ColumnTransformerEnc(dataset.descriptor)
ref_value = dataset.df.iloc[0].values[:-1]
encoded = tabular_enc.encode([ref_value])
decoded = tabular_enc.decode(encoded)

print(f"Original value: {ref_value}")
print(f"Encoded value: {encoded}")
print(f"Decoded value: {decoded}")

Original value: [5.1 3.5 1.4 0.2]
Encoded value: [[5.1 3.5 1.4 0.2]]
Decoded value: [[5.1 3.5 1.4 0.2]]


#### Neighborhood generation
Now that we are able to encode and decode the dataset, we can generate the neighborhood of the instance to be explained. To personalize the neighborhood generation process, we create now a Genetic Generator, using the class `GeneticGenerator`. The neighborhood is a synthetic dataset that is created by generating random instances around the instance to be explained, refined with a genetic algorithm to obtain a more dense and compact neighborhood.

In [33]:
x = dataset.df.iloc[num_row][:-1]
x

sepal length (cm)    5.4
sepal width (cm)     3.7
petal length (cm)    1.5
petal width (cm)     0.2
Name: 10, dtype: object

In [34]:
from lore_sa.neighgen import RandomGenerator
from lore_sa.neighgen import GeneticGenerator

num_row = 10
x = dataset.df.iloc[num_row][:-1]
z = tabular_enc.encode([x.values])[0] # remove the class feature from the input instance

# gen = RandomGenerator(bbox=bbox, dataset=dataset, encoder=tabular_enc, ocr=0.1)
# neighbour = gen.generate(z, 100, dataset.descriptor, tabular_enc)

gen = GeneticGenerator(bbox=bbox, dataset=dataset, encoder=tabular_enc, ocr=0.1)
neighbour = gen.generate(z, 100, dataset.descriptor, tabular_enc)

print('Neighborhood', neighbour)

gen	nevals	avg     	min     	max     
0  	50    	0.993307	0.993307	0.993307
1  	29    	0.992757	0.965781	0.993307
2  	26    	0.993252	0.99082 	0.993307
3  	24    	0.993297	0.992826	0.993307
4  	25    	0.992458	0.950853	0.993307
5  	31    	0.983268	0.497174	0.993307
6  	32    	0.992282	0.974313	0.993307
7  	27    	0.992774	0.973903	0.993307
8  	26    	0.992652	0.963965	0.993307
9  	30    	0.992342	0.961631	0.993307
10 	25    	0.982518	0.4549  	0.993307
11 	35    	0.993273	0.991938	0.993307
12 	34    	0.992948	0.977422	0.993307
13 	31    	0.993049	0.986852	0.993307
14 	36    	0.992151	0.93868 	0.993307
15 	38    	0.983338	0.497784	0.993307
16 	16    	0.993275	0.992353	0.993307
17 	22    	0.982971	0.497519	0.993307
18 	37    	0.983319	0.498198	0.993307
19 	27    	0.993071	0.986967	0.993307
20 	27    	0.993233	0.990929	0.993307
21 	26    	0.99303 	0.987356	0.993307
22 	24    	0.993045	0.984925	0.993307
23 	38    	0.983142	0.492135	0.993307
24 	27    	0.972819	0.489803	0.993307
25 	22    	0

## surrogate model

The `surrogate` submodule creates a classifier and provides the methods to extract its corresponding classification rules. Once trained the surrogate, we extract a set of rules that explains why the surrogate model classifies the instance in a certain way. 

In the following example, the instance `x` has been classified as an adult with income less than 50k. The surrogate model has used the features `capital-gain`, `capital-loss`, `marital-status`, and `native-country`. 

In [35]:
from lore_sa.surrogate import DecisionTreeSurrogate
# decode the neighborhood to be labeled by the blackbox model
neighb_train_X = tabular_enc.decode(neighbour)
neighb_train_y = bbox.predict(neighb_train_X)
# encode the target class to the surrogate model
neighb_train_yz = tabular_enc.encode_target_class(neighb_train_y.reshape(-1, 1)).squeeze()

dt = DecisionTreeSurrogate()
dt.train(neighbour, neighb_train_yz)

DecisionTreeClassifier()

In [36]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # remove the class feature from the input instance
z = tabular_enc.encode([x.values])[0]
rule = dt.get_rule(z, tabular_enc)
print('rule', rule)
crules, deltas = dt.get_counterfactual_rules(z, neighbour, neighb_train_yz, tabular_enc)
print('\n crules')
for c in crules:
    print(c)


rule premises:
petal length (cm) <= 2.0451077818870544 
consequence: target = setosa

 crules
premises:
petal length (cm) <= 5.24004602432251
petal length (cm) > 2.0451077818870544 
consequence: target = versicolor
